In [1]:


from landcover import LandCoverProportions

lc_path         = os.path.join(c.DATA_DIR, c.COPERNICUS_LC_KEY)
boundaries_path = os.path.join(c.DATA_DIR, "wb_countries_admin0_10m.zip")

# Load secrets
with open(c.SECRETS_PATH, 'r') as f:
    secrets = yaml.safe_load(f)

aws_access_key_id     = secrets['aws_access_key_id']
aws_secret_access_key = secrets['aws_secret_access_key']

s3 = boto3.resource('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

# Attempt to download files from S3

If the file can't be found on S3, attempt to upload it from the local filesystem.

In [2]:
for filepath in [lc_path, boundaries_path]:
    if os.path.exists(filepath):
        print(f"{filepath} already exists, skipping download")
    else:
        try:
            filename = os.path.basename(filepath)
            # Download the file from s3
            print(f"Downloading {filename} from s3")
            s3.Bucket(c.S3_BUCKET).download_file('raw/' + filename, filepath)
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                print("The object does not exist.")

                # Attempt to upload the file to s3
                try:
                    print(f"Uploading {filename} to s3")
                    s3.Bucket(c.S3_BUCKET).upload_file(filepath, filename)
                except botocore.exceptions.ClientError as e:
                    print(e)
                    raise e

/Users/chriskrapu/Dropbox/projects/vectorgeo/data/PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-map_EPSG-4326.tif already exists, skipping download
/Users/chriskrapu/Dropbox/projects/vectorgeo/data/wb_countries_admin0_10m.zip already exists, skipping download


# Create geometries for sampling from valid locations

In [3]:
boundaries_gdf = gpd.read_file('data/WB_countries_Admin0_10m/WB_countries_Admin0_10m.shp')
world_gdf = gpd.GeoDataFrame(geometry=[boundaries_gdf.unary_union], crs=boundaries_gdf.crs)

# Load raster into memory

In [4]:
lcp = LandCoverProportions(lc_path, world_gdf)

# Sampling / zonal statistics loop

In [5]:
def upload_df_s3(df, filename, vpath):
    print(f"Beginning to upload {len(proportions_df)} rows to {filename}...")
    
    df.to_parquet(filename)
    s3.meta.client.upload_file(filename, c.S3_BUCKET, f'{vpath}/{filename}')
    os.remove(filename)

for i in trange(10_000):
    try:
        data = list(lcp.generate_proportions(10_000, c.LC_K_RING_SETS, c.LC_H3_RES))
        h3_idx, records = [x[0] for x in data], [x[1] for x in data]
        proportions_df = pd.DataFrame(records, index=h3_idx)
        file_id = abs(hash(str(time.time()))) % (10 ** 6)
        filename = f'lulc-vector-res{c.LC_H3_RES}-{file_id}.parquet'
        upload_df_s3(proportions_df, filename, c.LC_VPATH
)
    except KeyboardInterrupt:
        print(f"Execution interrupted with {len(data)} records")
        break


  0%|          | 0/10000 [00:00<?, ?it/s]/Users/chriskrapu/Dropbox/projects/vectorgeo/vg-venv/lib/python3.9/site-packages/rasterstats/io.py:328: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(
  0%|          | 0/10000 [58:42<?, ?it/s]


NameError: name 'data' is not defined